In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import cloudknot as ck
import diff_classifier.knotlets as kn
import numpy as np

In [2]:
folder = '01_18_Experiment'

missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}".format("%04d" % i))

In [ ]:
missing

In [3]:
to_track = {}
knot = {}
result_futures = {}
start_knot = 130

pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
folder = '01_18_Experiment'

hemis = ["R", "L"]

github_installs=('https://github.com/ccurtis7/diff_classifier.git')
my_image = ck.DockerImage(func=kn.download_split_track_msds, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

cut1 = docker_string.find('# Create a default user')
cut2 = docker_string[cut1:].find('\n\n') + cut1
new_docker = docker_string[0:cut1]+docker_string[cut2:]
docker_overwrite = open(my_image.docker_path, 'w')
docker_overwrite.write(new_docker)
docker_overwrite.close()

req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-2]+'28'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

for pup in pups:
    for hemi in hemis:

            run_name = '{}_{}'.format(pup, hemi)
            to_track[run_name] = []
            for slic in slices:
                for vid in range(0, 15):
                            prefix = "{}_{}_{}_{}".format(pup, slic, hemi, "%04d" % vid)
                            if not prefix in missing:
                                to_track[run_name].append(prefix)

            test_length = len(to_track[run_name])
            print('Number of nodes to be loaded: {}'.format(test_length))
            
            knot[run_name] = ck.Knot(name='download_and_track_{}'.format(start_knot),
                           docker_image = my_image,
                           memory = 120000,
                           pars_policies=('AmazonS3FullAccess',))
            result_futures[run_name] = knot[run_name].map(to_track[run_name])
            start_knot = start_knot + 1
            print('Next knot name: {}'.format(start_knot))

Number of nodes to be loaded: 40
Next knot name: 131
Number of nodes to be loaded: 45
Next knot name: 132
Number of nodes to be loaded: 45
Next knot name: 133
Number of nodes to be loaded: 40
Next knot name: 134
Number of nodes to be loaded: 45
Next knot name: 135
Number of nodes to be loaded: 30
Next knot name: 136


In [ ]:
for key in knot:
    knot[key].clobber()

In [ ]:
knot